In [1]:
#MAE_test =  metrics.mean_absolute_error(y_test, y_pred)

In [2]:
import sys
!{sys.executable} -m pip install tabulate

In [60]:
#nessicary packages
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date
import time
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from tabulate import tabulate
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.svm import SVR
from graphviz import Digraph
import graphviz

In [4]:
#get sample data
file = "trip39A.txt"

df = pd.read_csv(file, delimiter = ",")

In [5]:
#drop redundant data
redudantRows = ["DATASOURCE","BASIN","TENDERLOT","SUPPRESSED","JUSTIFICATIONID","LASTUPDATE","NOTE","Unnamed: 0"]

df.drop(columns=redudantRows, inplace=True)
df.shape

(149, 9)

In [6]:
#test target features
df["PLANNEDTIME_TOTAL"] = df["PLANNEDTIME_ARR"] - df["PLANNEDTIME_DEP"]
df["ACTUALTIME_TOTAL"] = df["ACTUALTIME_ARR"] - df["ACTUALTIME_DEP"]
df["DELAY"] = df["ACTUALTIME_TOTAL"] - df["PLANNEDTIME_TOTAL"]

df.shape

(149, 12)

In [7]:
#Drop NA Values
df.dropna(inplace = True)
df.shape

(127, 12)

In [8]:
x = list(dict.fromkeys(df["ROUTEID"].tolist()))
print(x)

['39A_40', '39A_43', '39A_42']


In [9]:
len(x)

3

In [10]:
df["DIRECTION"] = df["DIRECTION"].astype("category")
df["TRIPID"] = df["TRIPID"].astype("category")
df['LINEID'] = df['LINEID'].astype("category")
df['ROUTEID'] = df['ROUTEID'].astype("category")

In [11]:
#get hour and minutes
df["hours"] = df["ACTUALTIME_DEP"]//3600
df["minutes"] = ((df["ACTUALTIME_DEP"]/3600 - df["ACTUALTIME_DEP"]//3600)*60)//1
df.dtypes

DAYOFSERVICE           object
TRIPID               category
LINEID               category
ROUTEID              category
DIRECTION            category
PLANNEDTIME_ARR         int64
PLANNEDTIME_DEP         int64
ACTUALTIME_ARR        float64
ACTUALTIME_DEP        float64
PLANNEDTIME_TOTAL       int64
ACTUALTIME_TOTAL      float64
DELAY                 float64
hours                 float64
minutes               float64
dtype: object

In [12]:
df

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,PLANNEDTIME_TOTAL,ACTUALTIME_TOTAL,DELAY,hours,minutes
0,07-FEB-18 00:00:00,6246028,39A,39A_40,1,49276,44400,50038.0,44382.0,4876,5656.0,780.0,12.0,19.0
1,07-FEB-18 00:00:00,6261203,39A,39A_43,2,39535,34800,40249.0,34984.0,4735,5265.0,530.0,9.0,43.0
2,07-FEB-18 00:00:00,6251013,39A,39A_40,1,71287,66600,72534.0,66602.0,4687,5932.0,1245.0,18.0,30.0
3,26-JUN-18 00:00:00,7104333,39A,39A_43,2,40074,35400,40759.0,35333.0,4674,5426.0,752.0,9.0,48.0
4,26-JUN-18 00:00:00,7104336,39A,39A_40,1,58316,53400,58367.0,53393.0,4916,4974.0,58.0,14.0,49.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,18-FEB-18 00:00:00,6264923,39A,39A_43,2,46163,42000,46657.0,42023.0,4163,4634.0,471.0,11.0,40.0
145,18-FEB-18 00:00:00,6274917,39A,39A_43,2,42863,38700,43234.0,38743.0,4163,4491.0,328.0,10.0,45.0
146,18-FEB-18 00:00:00,6271772,39A,39A_43,2,41063,36900,41036.0,36945.0,4163,4091.0,-72.0,10.0,15.0
147,18-FEB-18 00:00:00,6263988,39A,39A_43,2,44663,40500,44979.0,40500.0,4163,4479.0,316.0,11.0,15.0


In [13]:
#df["month"] = df["DATE"]

In [14]:
#df["month"]

In [15]:
#format date column correctly

In [16]:
#create datetime column with all the data
df["month"] = np.nan
df["day"] = np.nan
df['DATE'] = np.nan

for i in range(df.shape[0]):
        df['DATE'].iloc[i] = datetime.strptime(df['DAYOFSERVICE'].iloc[i], '%d-%b-%y 00:00:00')
        df["DATE"].iloc[i]= df["DATE"].iloc[i].replace(hour=int(df["hours"].iloc[i]), minute=int(df["minutes"].iloc[i]))
        df["month"].iloc[i] = df.DATE.iloc[i].month

df["month"] = df["month"].astype("int64")        

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [17]:
#Get weather data
dfWeather = pd.read_csv("2018_historic_weather_3.txt", delimiter = "\t",  parse_dates=False)
dfWeather.dt = dfWeather.dt.astype('datetime64[ns]')
#dfWeather.dt

dfWeather.dtypes

dt                     datetime64[ns]
dt_iso                         object
timezone                        int64
city_name                      object
lat                           float64
lon                           float64
temp                          float64
feels_like                    float64
temp_min                      float64
temp_max                      float64
pressure                        int64
sea_level                     float64
grnd_level                    float64
humidity                        int64
wind_speed                    float64
wind_deg                        int64
rain_1h                       float64
rain_3h                       float64
snow_1h                       float64
snow_3h                       float64
clouds_all                      int64
weather_id                      int64
weather_main                   object
weather_description            object
weather_icon                   object
dtype: object

In [18]:
#create propper weather columns
dfWeather['dates'] = np.nan

for i in range(8800):
    dfWeather['dates'].iloc[i] = datetime.strptime(dfWeather['dt_iso'].iloc[i], '%Y-%m-%d %H:%M:%S +0000 UTC')

In [19]:
#select features that may have proise with tarket outcome
dfWeather["DATE"] = dfWeather['dates']
dfweather1 = dfWeather[["DATE","temp","feels_like","temp_min","temp_max","pressure","humidity","weather_main"]]

In [20]:
dfweather1.DATE = dfweather1.DATE.astype('datetime64[ns]')
dfweather1.dtypes

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


DATE            datetime64[ns]
temp                   float64
feels_like             float64
temp_min               float64
temp_max               float64
pressure                 int64
humidity                 int64
weather_main            object
dtype: object

In [21]:
#create cleaned wether data set
dfWeather_main = pd.get_dummies(dfweather1['weather_main'])
dfWeather_cleaned = pd.concat([dfweather1[["DATE","temp","feels_like","temp_min","temp_max","pressure","humidity"]],pd.get_dummies(dfweather1['weather_main'])],axis=1)

dfWeather_cleaned["DATES"] = dfWeather_cleaned["DATE"]
dfWeather_cleaned.DATES = dfWeather_cleaned.DATES.astype('datetime64[ns]')
dfWeather_cleaned.dtypes

DATE          datetime64[ns]
temp                 float64
feels_like           float64
temp_min             float64
temp_max             float64
pressure               int64
humidity               int64
Clear                  uint8
Clouds                 uint8
Drizzle                uint8
Fog                    uint8
Mist                   uint8
Rain                   uint8
Smoke                  uint8
Snow                   uint8
DATES         datetime64[ns]
dtype: object

In [22]:
#df.DATES.astype('object')
df_final = df[["TRIPID","DATE","DIRECTION","PLANNEDTIME_TOTAL","ACTUALTIME_TOTAL","DELAY","LINEID","ROUTEID","hours","month","day"]]
df_final.DATE = df_final.DATE.astype('datetime64[ns]')
df_final.dtypes

TRIPID                     category
DATE                 datetime64[ns]
DIRECTION                  category
PLANNEDTIME_TOTAL             int64
ACTUALTIME_TOTAL            float64
DELAY                       float64
LINEID                     category
ROUTEID                    category
hours                       float64
month                         int64
day                         float64
dtype: object

# Total Data Set

In [23]:
df_total = pd.merge_asof(df_final.sort_values("DATE"), dfWeather_cleaned, left_on="DATE", right_on="DATE")

In [24]:
for i in range(df_total.shape[0]):
        df_total["day"].iloc[i] = df_total["DATE"].iloc[i].dayofweek       

df_total["day"] = df_total["day"].astype("int64")  

In [25]:
#days of the week:
df_total["weekend"] = np.logical_or(df_total["day"]== 5, df_total["day"]== 6)
df_total["weekend"].replace({False:0,True:1},inplace=True)
df_total["weekday"] = np.logical_or(df_total["day"]>5, df_total["weekend"]== 0)
df_total["weekday"].replace({False:0,True:1},inplace=True)

In [26]:
df_total["DATE"].iloc[1].date()

datetime.date(2018, 2, 7)

In [27]:
#This is to check if the date is a public holiday
# There was no pulblic holiday data in the sample data

ny = pd.Timestamp("2-7-2018")

publicHoliday = [pd.Timestamp("1-1-2018"), pd.Timestamp("3-17-2018"), pd.Timestamp("12-25-2018"), pd.Timestamp("12-26-2018")]

print(ny.date() in publicHoliday)

False


In [28]:
#convert directions to binary column
df_total.DIRECTION.replace(to_replace=2, value=0, inplace=True)

In [29]:
df_total.head()

,TRIPID,DATE,DIRECTION,PLANNEDTIME_TOTAL,ACTUALTIME_TOTAL,DELAY,LINEID,ROUTEID,hours,month,...,Clouds,Drizzle,Fog,Mist,Rain,Smoke,Snow,DATES,weekend,weekday
0,6261203,2018-02-07 09:43:00,0,4735,5265.0,530.0,39A,39A_43,9.0,2,...,1,0,0,0,0,0,0,2018-02-07 09:00:00,0,1
1,6246028,2018-02-07 12:19:00,1,4876,5656.0,780.0,39A,39A_40,12.0,2,...,1,0,0,0,0,0,0,2018-02-07 12:00:00,0,1
2,6251013,2018-02-07 18:30:00,1,4687,5932.0,1245.0,39A,39A_40,18.0,2,...,1,0,0,0,0,0,0,2018-02-07 18:00:00,0,1
3,6274915,2018-02-18 08:19:00,0,3571,3920.0,349.0,39A,39A_43,8.0,2,...,1,0,0,0,0,0,0,2018-02-18 08:00:00,1,1
4,6271772,2018-02-18 10:15:00,0,4163,4091.0,-72.0,39A,39A_43,10.0,2,...,0,0,0,0,1,0,0,2018-02-18 10:00:00,1,1


In [30]:
#add rush hour
df_total["rush_hour"] = np.logical_or(df_total["hours"]== 7.0, df_total["day"]== 8.0)
df_total["rush_hour"] = np.logical_or(df_total["rush_hour"] == True, df_total["day"]== 16.0)
df_total["rush_hour"] = np.logical_or(df_total["rush_hour"] == True, df_total["day"]== 17.0)
df_total["rush_hour"].replace({False:0,True:1},inplace=True)

In [31]:
df_total_origional = df_total
#df_total_in = df_total[df_total["DIRECTION"]==1]
#df_total_out = df_total[df_total["DIRECTION"]==0]

#df_total = df_total_in

In [32]:
df_total.corr()[["DELAY","ACTUALTIME_TOTAL"]].sort_values(by=['DELAY'], ascending=False)

,DELAY,ACTUALTIME_TOTAL
DELAY,1.000000,0.700393
ACTUALTIME_TOTAL,0.700393,1.000000
rush_hour,0.169843,0.082770
day,0.131843,-0.094949
pressure,0.122984,0.117790
Mist,0.111460,-0.001347
hours,0.051162,-0.151248
Drizzle,0.049660,-0.089720
weekend,0.037114,-0.233964
PLANNEDTIME_TOTAL,0.034620,0.737577


In [33]:
#keep feels like
#drop min_temp, max_temp, temp

In [34]:
df_total.columns

Index(['TRIPID', 'DATE', 'DIRECTION', 'PLANNEDTIME_TOTAL', 'ACTUALTIME_TOTAL',
       'DELAY', 'LINEID', 'ROUTEID', 'hours', 'month', 'day', 'temp',
       'feels_like', 'temp_min', 'temp_max', 'pressure', 'humidity', 'Clear',
       'Clouds', 'Drizzle', 'Fog', 'Mist', 'Rain', 'Smoke', 'Snow', 'DATES',
       'weekend', 'weekday', 'rush_hour'],
      dtype='object')

In [35]:
#Direction, 1 = outbound , 0 = inbound

In [36]:
# Monday = 0
# Tuesday = 1
# Wednesday = 2
# Thursday = 3
# Friday = 4
# Saturday = 5
# Sunday = 6

# What Effects Delay:
- Direction
- Pressure
- Mist
- Rain
- Humidity


# What effects Actualtime_Total:
- Direction
- Pressure
- Hours
- PlannedTime
- Clouds
- Rain
- Humidity
- Month

TRIPID was not included in this as it is a unique identifier for each trip, and there should not be any correlation between the unique identifier for the dataset and the target outcome. 

In [37]:
features = ["DIRECTION","pressure","hours","PLANNEDTIME_TOTAL","Clouds","Rain","humidity","month","weekday", "rush_hour"]

#features = ["DIRECTION","pressure","Mist","Rain","humidity"]

In [38]:
df_total[features]

,DIRECTION,pressure,hours,PLANNEDTIME_TOTAL,Clouds,Rain,humidity,month,weekday,rush_hour
0,0,1024,9.0,4735,1,0,93,2,1,0
1,1,1023,12.0,4876,1,0,87,2,1,0
2,1,1019,18.0,4687,1,0,87,2,1,0
3,0,1017,8.0,3571,1,0,93,2,1,0
4,0,1017,10.0,4163,0,1,93,2,1,0
...,...,...,...,...,...,...,...,...,...,...
122,0,993,9.0,5190,1,0,82,10,1,0
123,0,993,9.0,5190,1,0,82,10,1,0
124,0,993,9.0,5190,1,0,82,10,1,0
125,0,998,18.0,5121,1,0,82,10,1,0


In [39]:
df_total.dtypes

for i in df_total.columns:
    if df_total[i].dtype == "uint8":
        df_total[i] = df_total[i].astype("int64")

In [40]:
df_total.dtypes

TRIPID                     category
DATE                 datetime64[ns]
DIRECTION                  category
PLANNEDTIME_TOTAL             int64
ACTUALTIME_TOTAL            float64
DELAY                       float64
LINEID                     category
ROUTEID                    category
hours                       float64
month                         int64
day                           int64
temp                        float64
feels_like                  float64
temp_min                    float64
temp_max                    float64
pressure                      int64
humidity                      int64
Clear                         int64
Clouds                        int64
Drizzle                       int64
Fog                           int64
Mist                          int64
Rain                          int64
Smoke                         int64
Snow                          int64
DATES                datetime64[ns]
weekend                       int64
weekday                     

In [41]:
#df_total["day"] = df_total["DATE"].dayofweek

In [42]:
#df_total_orig #= df_total

#df_total = df_total_orig[df_total_orig["DIRECTION"]==0]

#df_total = df_total[df_total["ROUTEID"]=="39A_43"]

In [43]:
df_total

,TRIPID,DATE,DIRECTION,PLANNEDTIME_TOTAL,ACTUALTIME_TOTAL,DELAY,LINEID,ROUTEID,hours,month,...,Drizzle,Fog,Mist,Rain,Smoke,Snow,DATES,weekend,weekday,rush_hour
0,6261203,2018-02-07 09:43:00,0,4735,5265.0,530.0,39A,39A_43,9.0,2,...,0,0,0,0,0,0,2018-02-07 09:00:00,0,1,0
1,6246028,2018-02-07 12:19:00,1,4876,5656.0,780.0,39A,39A_40,12.0,2,...,0,0,0,0,0,0,2018-02-07 12:00:00,0,1,0
2,6251013,2018-02-07 18:30:00,1,4687,5932.0,1245.0,39A,39A_40,18.0,2,...,0,0,0,0,0,0,2018-02-07 18:00:00,0,1,0
3,6274915,2018-02-18 08:19:00,0,3571,3920.0,349.0,39A,39A_43,8.0,2,...,0,0,0,0,0,0,2018-02-18 08:00:00,1,1,0
4,6271772,2018-02-18 10:15:00,0,4163,4091.0,-72.0,39A,39A_43,10.0,2,...,0,0,0,1,0,0,2018-02-18 10:00:00,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,8024541,2018-10-11 09:19:00,0,5190,5557.0,367.0,39A,39A_43,9.0,10,...,0,0,0,0,0,0,2018-10-11 09:00:00,0,1,0
123,8025109,2018-10-11 09:29:00,0,5190,5587.0,397.0,39A,39A_43,9.0,10,...,0,0,0,0,0,0,2018-10-11 09:00:00,0,1,0
124,8020475,2018-10-11 09:51:00,0,5190,5419.0,229.0,39A,39A_43,9.0,10,...,0,0,0,0,0,0,2018-10-11 09:00:00,0,1,0
125,8023269,2018-10-11 18:21:00,0,5121,6079.0,958.0,39A,39A_43,18.0,10,...,0,0,0,0,0,0,2018-10-11 18:00:00,0,1,0


In [44]:
#x_train, x_test, y_train, y_test = train_test_split(df_total[features], df_total["significant_delay"], test_size=0.3, random_state=0)
x_train, x_test, y_train, y_test = train_test_split(df_total[features], df_total["ACTUALTIME_TOTAL"], test_size=0.3, random_state=0)

In [45]:
# building the models
# Model 1
logreg = LogisticRegression().fit(x_train[features], y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [46]:
#logistic regression results
logreg_table = {"Features":features, "Coefficient": logreg.coef_[0]}
print("\nIntercept: \n", logreg.intercept_,"\n")

print(tabulate(logreg_table, headers='keys'))


Intercept: 
 [ 2.26363921e-03  9.54033544e-04  9.54033544e-04  8.80627523e-04
  3.33461591e-04  2.47364846e-04  3.36445327e-04  4.24979231e-04
  7.92585013e-04  5.66169334e-04  3.36445327e-04 -2.68740449e-04
  1.71554025e-04  4.24979231e-04 -2.68740449e-04  3.36445327e-04
  6.76780411e-05 -2.68740449e-04  2.19371955e-04  3.87311752e-04
  2.48647730e-04  2.47364846e-04  2.48647730e-04  2.85285866e-04
  3.92533904e-04  3.79431242e-04  6.93789539e-05  9.33395271e-05
 -5.68710900e-04 -2.68757185e-04 -1.82796831e-04 -3.50272664e-04
  3.85097795e-04 -2.68757185e-04 -2.68740449e-04  5.49233851e-05
  3.54888355e-05  2.93782963e-04  1.71554025e-04 -3.11502283e-04
  6.93789539e-05  2.55992055e-04 -5.11577551e-05  2.51283577e-04
  3.36526513e-04  1.71439490e-04 -2.68740449e-04  5.49233851e-05
  7.97787348e-05 -1.54837451e-04  3.79431242e-04  9.34737842e-05
 -7.40237266e-04 -5.45992064e-04 -2.68757185e-04  9.34737842e-05
  2.56007299e-06 -5.68085821e-04  1.71554025e-04  1.71439490e-04
 -2.6875718

In [47]:
#train predictions
logreg_predictions_train = (logreg.predict(x_train[features]))
MAE_train_log =  metrics.mean_absolute_error(y_train, logreg_predictions_train)
MSE_train_log = metrics.mean_squared_error(y_train, logreg_predictions_train)
RMSE_train_log = MSE_train_log**0.5

print("Train MAE:", MAE_train_log)
print("Train MSE:", MSE_train_log)
print("Train RMSE:", RMSE_train_log)

Train MAE: 297.57954545454544
Train MSE: 263615.76136363635
Train RMSE: 513.4352552792186


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)


In [48]:
#test predictions
logreg_predictions_test = (logreg.predict(x_test[features]))
MAE_test_log =  metrics.mean_absolute_error(y_test, logreg_predictions_test)
MSE_test_log = metrics.mean_squared_error(y_test, logreg_predictions_test)
RMSE_test_log = MSE_test_log**0.5

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)


In [49]:
MAE_test_log
print("Test MAE:", MAE_test_log)
print("Test MSE:", MSE_test_log)
print("Test RMSE:", RMSE_test_log)

Test MAE: 585.5384615384615
Test MSE: 578466.4102564103
Test RMSE: 760.5697931527457


# Decision Trees

In [50]:
rfc = RandomForestClassifier(n_estimators=100, max_features='auto', oob_score=True, random_state=0)
rfc.fit(x_train[features], y_train)

RandomForestClassifier(oob_score=True, random_state=0)

In [51]:
#see v2 document

# Linear Model

In [52]:
linreg = LinearRegression().fit(x_train[features], y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)


In [53]:
linreg_table = {"Features":features, "Coefficient": linreg.coef_}
print("\nIntercept: \n", linreg.intercept_,"\n")

print(tabulate(linreg_table, headers='keys'))


Intercept: 
 -6182.009008783939 

Features             Coefficient
-----------------  -------------
DIRECTION               84.2231
pressure                 6.38556
hours                   13.1826
PLANNEDTIME_TOTAL        1.19198
Clouds                 -80.6801
Rain                  -255.955
humidity                -6.20104
month                  -52.4342
weekday                  0
rush_hour              224.533


In [74]:
linreg_predictions_train = (linreg.predict(x_train[features]))
MAE_train_lin =  metrics.mean_absolute_error(y_train, linreg_predictions_train)
MSE_train_lin = metrics.mean_squared_error(y_train, linreg_predictions_train)
RMSE_train_lin = MSE_train_lin**0.5

print("Train MAE:", MAE_train_lin)
print("Train MSE:", MSE_train_lin)
print("Train RMSE:", RMSE_train_lin)

Train MAE: 384.2604437988876
Train MSE: 251941.63769285005
Train RMSE: 501.93788230502196


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)


In [75]:
linreg_predictions_test = (linreg.predict(x_test[features]))
MAE_test_lin =  metrics.mean_absolute_error(y_test, linreg_predictions_test)
MSE_test_lin = metrics.mean_squared_error(y_test, linreg_predictions_test)
RMSE_test_lin = MSE_test_lin**0.5

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)


In [76]:
print("Test MAE:", MAE_test_lin)
print("Test MSE:", MSE_test_lin)
print("Test RMSE:", RMSE_test_lin)

Test MAE: 408.8298439056381
Test MSE: 282698.02100783616
Test RMSE: 531.6935404985057


# SVM

In [78]:
# 
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_lin = SVR(kernel='linear', C=1e3)
svr_poly = SVR(kernel='poly', C=1e3, degree=2)
svm_rbf = svr_rbf.fit(x_train[features], y_train)
svm_lin = svr_lin.fit(x_train[features], y_train)
svm_poly = svr_poly.fit(x_train[features], y_train)

In [79]:
#Radial Basis Function
#train
svm_rbf_predictions_train = svm_rbf.predict(x_train[features])
MAE_train_rbf =  metrics.mean_absolute_error(y_train, svm_rbf_predictions_train)
MSE_train_rbf = metrics.mean_squared_error(y_train, svm_rbf_predictions_train)
RMSE_train_rbf = MSE_train_rbf**0.5

print("Train MAE:", MAE_train_rbf)
print("Train MSE:", MSE_train_rbf)
print("Train RMSE:", RMSE_train_rbf,"\n")

#test
svm_rbf_predictions_test = (svm_rbf.predict(x_test[features]))
MAE_test_rbf =  metrics.mean_absolute_error(y_test, svm_rbf_predictions_test)
MSE_test_rbf = metrics.mean_squared_error(y_test, svm_rbf_predictions_test)
RMSE_test_rbf = MSE_test_rbf**0.5

print("Test MAE:", MAE_test_rbf)
print("Test MSE:", MSE_test_rbf)
print("Test RMSE:", RMSE_test_rbf)
 
#svm_poly_train = 

Train MAE: 249.96177578020843
Train MSE: 213298.59533191053
Train RMSE: 461.84260883109357 

Test MAE: 524.8632222168412
Test MSE: 357786.8724609786
Test RMSE: 598.1528838524299


In [80]:
#Linear model (SVM)
#train
svm_lin_predictions_train = svm_lin.predict(x_train[features])
MAE_train_lin =  metrics.mean_absolute_error(y_train, svm_lin_predictions_train)
MSE_train_lin = metrics.mean_squared_error(y_train, svm_lin_predictions_train)
RMSE_train_lin = MSE_train_lin**0.5

print("Train MAE:", MAE_train_lin)
print("Train MSE:", MSE_train_lin)
print("Train RMSE:", RMSE_train_lin,"\n")

#test
svm_lin_predictions_test = (svm_lin.predict(x_test[features]))
MAE_test_lin =  metrics.mean_absolute_error(y_test, svm_lin_predictions_test)
MSE_test_lin = metrics.mean_squared_error(y_test, svm_lin_predictions_test)
RMSE_test_lin = MSE_test_lin**0.5

print("Test MAE:", MAE_test_lin)
print("Test MSE:", MSE_test_lin)
print("Test RMSE:", RMSE_test_lin)

Train MAE: 9362.185298067714
Train MSE: 117634998.81995569
Train RMSE: 10845.96693798924 

Test MAE: 8240.97334290902
Test MSE: 96975814.28656073
Test RMSE: 9847.629881680197


In [81]:
#Polynomial Model
#train
svm_poly_predictions_train = svm_poly.predict(x_train[features])
MAE_train_poly =  metrics.mean_absolute_error(y_train, svm_poly_predictions_train)
MSE_train_poly = metrics.mean_squared_error(y_train, svm_poly_predictions_train)
RMSE_train_poly = MSE_train_poly**0.5

print("Train MAE:", MAE_train_poly)
print("Train MSE:", MSE_train_poly)
print("Train RMSE:", RMSE_train_poly,"\n")

#test
svm_poly_predictions_test = (svm_poly.predict(x_test[features]))
MAE_test_poly =  metrics.mean_absolute_error(y_test, svm_poly_predictions_test)
MSE_test_poly = metrics.mean_squared_error(y_test, svm_poly_predictions_test)
RMSE_test_poly = MSE_test_lin**0.5

print("Test MAE:", MAE_test_poly)
print("Test MSE:", MSE_test_poly)
print("Test RMSE:", RMSE_test_poly)

Train MAE: 443.26660101485976
Train MSE: 327746.4838363303
Train RMSE: 572.4914705358765 

Test MAE: 454.3561884433616
Test MSE: 298424.8076551526
Test RMSE: 9847.629881680197


In [57]:
df_total

,TRIPID,DATE,DIRECTION,PLANNEDTIME_TOTAL,ACTUALTIME_TOTAL,DELAY,LINEID,ROUTEID,hours,month,...,Drizzle,Fog,Mist,Rain,Smoke,Snow,DATES,weekend,weekday,rush_hour
0,6261203,2018-02-07 09:43:00,0,4735,5265.0,530.0,39A,39A_43,9.0,2,...,0,0,0,0,0,0,2018-02-07 09:00:00,0,1,0
1,6246028,2018-02-07 12:19:00,1,4876,5656.0,780.0,39A,39A_40,12.0,2,...,0,0,0,0,0,0,2018-02-07 12:00:00,0,1,0
2,6251013,2018-02-07 18:30:00,1,4687,5932.0,1245.0,39A,39A_40,18.0,2,...,0,0,0,0,0,0,2018-02-07 18:00:00,0,1,0
3,6274915,2018-02-18 08:19:00,0,3571,3920.0,349.0,39A,39A_43,8.0,2,...,0,0,0,0,0,0,2018-02-18 08:00:00,1,1,0
4,6271772,2018-02-18 10:15:00,0,4163,4091.0,-72.0,39A,39A_43,10.0,2,...,0,0,0,1,0,0,2018-02-18 10:00:00,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,8024541,2018-10-11 09:19:00,0,5190,5557.0,367.0,39A,39A_43,9.0,10,...,0,0,0,0,0,0,2018-10-11 09:00:00,0,1,0
123,8025109,2018-10-11 09:29:00,0,5190,5587.0,397.0,39A,39A_43,9.0,10,...,0,0,0,0,0,0,2018-10-11 09:00:00,0,1,0
124,8020475,2018-10-11 09:51:00,0,5190,5419.0,229.0,39A,39A_43,9.0,10,...,0,0,0,0,0,0,2018-10-11 09:00:00,0,1,0
125,8023269,2018-10-11 18:21:00,0,5121,6079.0,958.0,39A,39A_43,18.0,10,...,0,0,0,0,0,0,2018-10-11 18:00:00,0,1,0


In [58]:
list(dict.fromkeys(df_total["ROUTEID"].tolist()))

['39A_43', '39A_40', '39A_42']